В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [3]:
!pip install opendatasets --upgrade --quiet
import opendatasets as od

dataset_url = 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu-course-c-3'

In [4]:
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yanakalashnikova
Your Kaggle Key: ··········


100%|██████████| 637k/637k [00:00<00:00, 501MB/s]


Extracting archive ./bank-customer-churn-prediction-dlu-course-c-3/bank-customer-churn-prediction-dlu-course-c-3.zip to ./bank-customer-churn-prediction-dlu-course-c-3


In [5]:
import pandas as pd

In [6]:
raw_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-3/train.csv")

In [10]:
# 1. Розділяємо на train та validation
train_df, val_df = split_train_val(raw_df, target_col="Exited", test_size=0.2, random_state=42)

# 2. Визначаємо ознаки (всі колонки крім цільової)
input_cols = [col for col in train_df.columns if col != "Exited"]

# 3. Витягуємо X та y
train_inputs, train_targets = separate_inputs_targets(train_df, input_cols, target_col="Exited")
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols, target_col="Exited")


In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [11]:
for col in train_inputs.select_dtypes(include='object').columns:
    train_inputs[col] = train_inputs[col].astype('category')
    val_inputs[col] = val_inputs[col].astype('category')

2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [12]:
!pip install xgboost

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Створення моделі
model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    use_label_encoder=False,
    eval_metric='logloss',
    tree_method='hist',
    enable_categorical=True,  # ← УВІМКНУТИ ЦЕ
    random_state=42
)

# Навчання
model.fit(train_inputs, train_targets)

# Передбачення й оцінка
train_preds_proba = model.predict_proba(train_inputs)[:, 1]
val_preds_proba = model.predict_proba(val_inputs)[:, 1]

# Обчислення AUROC
train_auc = roc_auc_score(train_targets, train_preds_proba)
val_auc = roc_auc_score(val_targets, val_preds_proba)

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:53:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Train AUROC: 0.9706
Validation AUROC: 0.9337


Висновок:
Отримані результати свідчать про те, що модель XGBoost добре навчена. Значення AUROC на тренувальній вибірці становить 0.9706, а на валідаційній — 0.9337, що вказує на високу якість узагальнення моделі.
Різниця між тренувальним та валідаційним AUROC незначна, тому можна зробити висновок, що перенавчання (overfitting) мінімальне.
Загалом, модель демонструє хороший баланс між bias та variance, і може бути використана для подальшого прогнозування відтоку клієнтів.

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [15]:
!pip install hyperopt

In [16]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# Простір пошуку
space = {
    'max_depth': hp.choice('max_depth', np.arange(3, 10, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.choice('n_estimators', np.arange(50, 300, 25, dtype=int)),
    'gamma': hp.uniform('gamma', 0, 5),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
}

# Функція цілі (мінімізуємо -AUROC)
def objective(params):
    model = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        tree_method='hist',
        enable_categorical=True,
        random_state=42,
        **params
    )
    model.fit(train_inputs, train_targets)
    val_preds = model.predict_proba(val_inputs)[:, 1]
    auc = roc_auc_score(val_targets, val_preds)
    return {'loss': -auc, 'status': STATUS_OK}

In [18]:
from numpy.random import default_rng

trials = Trials()

best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials,
    rstate=default_rng(42)  # ← правильний генератор
)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:59:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▌         | 1/20 [00:00<00:08,  2.26trial/s, best loss: -0.9287938130187255]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:59:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 15%|█▌        | 3/20 [00:01<00:07,  2.29trial/s, best loss: -0.9307833184717745]

...

 90%|█████████ | 18/20 [00:11<00:02,  1.03s/trial, best loss: -0.9360858083544825]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:59:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 95%|█████████▌| 19/20 [00:12<00:00,  1.00trial/s, best loss: -0.9360858083544825]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:59:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 20/20 [00:12<00:00,  1.58trial/s, best loss: -0.9360858083544825]
Best hyperparameters: {'colsample_bytree': np.float64(0.7531054068679127), 'gamma': np.float64(4.34677958142653), 'learning_rate': np.float64(0.033706613166125186), 'max_depth': np.int64(6), 'n_estimators': np.int64(0), 'subsample': np.float64(0.8495392973707541)}


In [20]:
print("Best hyperparameters:", best_params)

Best hyperparameters: {'colsample_bytree': np.float64(0.7531054068679127), 'gamma': np.float64(4.34677958142653), 'learning_rate': np.float64(0.033706613166125186), 'max_depth': 9, 'n_estimators': 50, 'subsample': np.float64(0.8495392973707541)}


In [21]:
best_params['max_depth'] = int(best_params['max_depth']) + 3
best_params['n_estimators'] = int(best_params['n_estimators']) * 25 + 50

In [22]:
final_clf = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    tree_method='hist',
    enable_categorical=True,
    random_state=42,
    **best_params
)

In [23]:
final_clf.fit(train_inputs, train_targets)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:01:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=np.float64(0.7531054068679127), device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric='logloss', feature_types=None,
              gamma=np.float64(4.34677958142653), grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=np.float64(0.033706613166125186), max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1300, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [24]:
from sklearn.metrics import roc_auc_score

train_preds_final = final_clf.predict_proba(train_inputs)[:, 1]
val_preds_final = final_clf.predict_proba(val_inputs)[:, 1]

train_auc_final = roc_auc_score(train_targets, train_preds_final)
val_auc_final = roc_auc_score(val_targets, val_preds_final)

print(f"Final Train AUROC: {train_auc_final:.4f}")
print(f"Final Validation AUROC: {val_auc_final:.4f}")

Final Train AUROC: 0.9771
Final Validation AUROC: 0.9320


Після використання бібліотеки HyperOpt для пошуку оптимальних гіперпараметрів, модель XGBoostClassifier була перенавчена з новими налаштуваннями.
Отримані результати демонструють:

Train AUROC покращився з 0.9706 до 0.9771

Validation AUROC залишився майже незмінним: 0.9337 → 0.9320

Це свідчить про те, що модель з оптимізованими параметрами трохи покращила точність на тренуванні, але узагальнення на валідації залишилось приблизно на тому ж рівні.
Отже, гіпероптимізація не призвела до значного переобучення і зберегла загальну якість моделі, що є хорошим результатом.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [25]:
!pip install lightgbm

In [26]:
# Індекси категоріальних ознак
cat_features = [i for i, col in enumerate(train_inputs.columns) if train_inputs[col].dtype.name == 'category']

In [28]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Створення моделі
lgb_clf = lgb.LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

# Навчання
lgb_clf.fit(
    train_inputs, train_targets,
    eval_set=[(val_inputs, val_targets)],
    eval_metric='auc',
    categorical_feature=cat_features,
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002132 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(max_depth=6, objective='binary', random_state=42)

In [29]:
train_preds_lgb = lgb_clf.predict_proba(train_inputs)[:, 1]
val_preds_lgb = lgb_clf.predict_proba(val_inputs)[:, 1]

train_auc_lgb = roc_auc_score(train_targets, train_preds_lgb)
val_auc_lgb = roc_auc_score(val_targets, val_preds_lgb)

print(f"LightGBM Train AUROC: {train_auc_lgb:.4f}")
print(f"LightGBM Validation AUROC: {val_auc_lgb:.4f}")

LightGBM Train AUROC: 0.9885
LightGBM Validation AUROC: 0.9305


Модель LightGBM продемонструвала дуже високу точність на тренувальній вибірці (AUROC = 0.9885), що є найвищим серед усіх моделей. Проте, на валідаційній вибірці вона показала AUROC = 0.9305, що трохи нижче, ніж у XGBoost (0.9337) та дуже близьке до оптимізованого XGBoost (0.9320).
Хоча LightGBM продемонстрував найвищу точність на train-наборі, оптимізований XGBoost залишився найкращим варіантом з точки зору якості узагальнення, а отже — є переважним для практичного застосування на нових даних.

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [35]:
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import numpy as np

# Простір параметрів
space_lgb = {
    'max_depth': hp.choice('max_depth', np.arange(3, 10, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.choice('n_estimators', np.arange(50, 300, 25, dtype=int)),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)
}

In [38]:
def objective_lgb(params):
    # Усунення 'verbose', якщо він прийшов з hyperopt
    params.pop('verbose', None)

    model = lgb.LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        random_state=42,
        **params
    )

    model.fit(
    train_inputs, train_targets,
    eval_set=[(val_inputs, val_targets)],
    eval_metric='auc',
    categorical_feature=cat_features
    )

    val_pred = model.predict_proba(val_inputs)[:, 1]
    auc = roc_auc_score(val_targets, val_pred)

    return {'loss': -auc, 'status': STATUS_OK}

In [39]:
from numpy.random import default_rng

trials_lgb = Trials()

best_lgb_params = fmin(
    fn=objective_lgb,
    space=space_lgb,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials_lgb,
    rstate=default_rng(42)
)

print("Best hyperparameters for LightGBM:", best_lgb_params)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001801 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large numb

In [40]:
# Розшифровуємо hp.choice
best_lgb_params['max_depth'] = int(best_lgb_params['max_depth']) + 3
best_lgb_params['n_estimators'] = int(best_lgb_params['n_estimators']) * 25 + 50

final_lgb_clf = lgb.LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    random_state=42,
    **best_lgb_params
)

final_lgb_clf.fit(train_inputs, train_targets,
                  eval_set=[(val_inputs, val_targets)],
                  eval_metric='auc',
                  categorical_feature=cat_features)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001125 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

LGBMClassifier(colsample_bytree=np.float64(0.7384474390538008),
               learning_rate=np.float64(0.0933333327574378), max_depth=3,
               n_estimators=175, objective='binary', random_state=42,
               subsample=np.float64(0.8011615018645928))

In [41]:
train_preds_final_lgb = final_lgb_clf.predict_proba(train_inputs)[:, 1]
val_preds_final_lgb = final_lgb_clf.predict_proba(val_inputs)[:, 1]

train_auc_final_lgb = roc_auc_score(train_targets, train_preds_final_lgb)
val_auc_final_lgb = roc_auc_score(val_targets, val_preds_final_lgb)

print(f"Final LightGBM Train AUROC: {train_auc_final_lgb:.4f}")
print(f"Final LightGBM Validation AUROC: {val_auc_final_lgb:.4f}")

Final LightGBM Train AUROC: 0.9665
Final LightGBM Validation AUROC: 0.9361


Висновки:

Якість моделі зросла після оптимізації гіперпараметрів.

Різниця між train і validation AUROC є прийнятною, що свідчить про хорошу генералізацію.

Hyperopt дозволив ефективно автоматизувати пошук найкращих параметрів, зокрема max_depth, learning_rate, n_estimators, subsample, colsample_bytree.

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [43]:
# 1. Завантаження тестового набору
test_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-3/test.csv")

# 2. Препроцесинг тестових даних (той самий, що й для train/val)
X_test = preprocess_new_data(test_df, input_cols, scaler, encoder)

# 3. Прогнозування
test_preds = final_lgb_clf.predict_proba(X_test)[:, 1]

# 4. Формування submission-файлу
# Якщо test_df має колонку 'id':
# submission = pd.DataFrame({ 'id': test_df['id'], 'target': test_preds })

# Якщо немає 'id', то створити свій:
submission = pd.DataFrame({
    'id': np.arange(len(test_preds)),
    'target': test_preds
})

# 5. Збереження у файл
submission.to_csv("submission.csv", index=False)

NameError: name 'preprocess_new_data' is not defined

Для фінального сабміту я обрала модель LightGBM з автоматичним підбором гіперпараметрів за допомогою Hyperopt, оскільки вона показала найвищу якість класифікації серед усіх експериментів:

Train AUC: 0.9665

Validation AUC: 0.9361

Ця модель має найменшу різницю між навчальною та валідаційною вибірками, що свідчить про хорошу узагальнювальну здатність. Крім того, LightGBM є обчислювально ефективним та стабільним алгоритмом для задач бустингу.

🟢 Отже, саме ця модель найкраще збалансувала точність і стійкість до перенавчання. Вона обґрунтовано була обрана для сабміту на Kaggle.